In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
dataset = pd.read_csv('raw.csv')
dataset = dataset.dropna()
dataset.head()

,Category,Title,Text
0,adulting-101,5_smart_ways_to_stretch_your_dollar_with_GrabFood,apps new features serve convenience value whet...
1,adulting-101,Adulting_101_People_around_me_are_job_hopping_...,adulthood invigorating stage life young people...
2,adulting-101,Airbnb_bans_security_cameras_inside_guest_homes,san francisco airbnb monday march 11 said bann...
3,adulting-101,As_Swiftonomics_sweeps_through_Singapore_small...,singapore swiftonomics swept singapore america...
4,adulting-101,At_least_50_hurt_as_LATAM_s_Boeing_787_to_Auck...,wellington least 50 people hurt boeing 787 ope...


In [ ]:
# Check for duplicates
dataset[dataset.Title == "5_smart_ways_to_stretch_your_dollar_with_GrabFood"]

,Category,Title,Text,CategoryId
0,adulting-101,5_smart_ways_to_stretch_your_dollar_with_GrabFood,apps new feature serve convenience value wheth...,0
59,big-read,5_smart_ways_to_stretch_your_dollar_with_GrabFood,apps new feature serve convenience value wheth...,1
117,commentary,5_smart_ways_to_stretch_your_dollar_with_GrabFood,apps new feature serve convenience value wheth...,2
176,gen-y-speaks,5_smart_ways_to_stretch_your_dollar_with_GrabFood,apps new feature serve convenience value wheth...,3
234,gen-z-speaks,5_smart_ways_to_stretch_your_dollar_with_GrabFood,apps new feature serve convenience value wheth...,4
285,singapore,5_smart_ways_to_stretch_your_dollar_with_GrabFood,apps new feature serve convenience value wheth...,5
343,voices,5_smart_ways_to_stretch_your_dollar_with_GrabFood,apps new feature serve convenience value wheth...,6
401,world,5_smart_ways_to_stretch_your_dollar_with_GrabFood,apps new feature serve convenience value wheth...,7


In [ ]:
# Step 1: Read the dataset
df = pd.read_csv('raw.csv')

# Step 2: Extract Features and Labels
title = df['Title'].values
features = df['Text'].values
labels = df['Category'].str.split('_').apply(lambda x: [x[0]])

# Step 3: Encode Labels
mlb = MultiLabelBinarizer()
binary_labels = mlb.fit_transform(labels)

# Step 4: Create Multi-label Dataset
multi_label_df = pd.DataFrame(binary_labels, columns=mlb.classes_)
multi_label_df['Text'] = features
multi_label_df['Title'] = title

columns = ["Title" , 'Text'] + list(mlb.classes_)
multi_label_df = multi_label_df[columns]

summarized_df = multi_label_df.groupby("Title").max()
summarized_df = summarized_df.dropna()
summarized_df[summarized_df.columns[1:]]

In [ ]:
summarized_df.head()

,Text,adulting-101,big-read,commentary,gen-y-speaks,gen-z-speaks,singapore,voices,world
Title,,,,,,,,,
5_smart_ways_to_stretch_your_dollar_with_GrabFood,apps new features serve convenience value whet...,1,1,1,1,1,1,1,1
Adulting_101_People_around_me_are_job_hopping_Am_I_missing_out_by_not_doing_the_same,adulthood invigorating stage life young people...,1,1,1,1,1,1,1,1
Airbnb_bans_security_cameras_inside_guest_homes,san francisco airbnb monday march 11 said bann...,1,1,1,1,1,1,1,1
As_Swiftonomics_sweeps_through_Singapore_small_businesses_say_concerts_a_boon_and_a_bane_for_them,singapore swiftonomics swept singapore america...,1,1,1,1,1,1,1,1
At_least_50_hurt_as_LATAM_s_Boeing_787_to_Auckland_just_dropped_mid_flight,wellington least 50 people hurt boeing 787 ope...,1,1,1,1,1,1,1,1


In [ ]:
summarized_df.to_csv('raw.csv')